In [ ]:
# Author - Kaustubh Chaudhuri
# With the help of Machine Learning Engineering Nanodegree from Udacity,
# I have developed this program- Kaustubh Chaudhuri
# Guidance of Dr. Sukgi Choi

from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline  
from keras.preprocessing import image                  
from tqdm import tqdm
from keras import backend as K




In [ ]:
# define function to load train, test, and validation datasets
def load_dataset(path, s):
    data = load_files(path)
    human_files = np.array(data['filenames'])
    human_targets = np_utils.to_categorical(np.array(data['target']), s)
    return human_files, human_targets

In [ ]:
#this is the number of individuals to be recognized
subjects = 5

# load train, test, and validation datasets
train_files, train_targets = load_dataset('humanImages/train', subjects)
valid_files, valid_targets = load_dataset('humanImages/valid', subjects)
test_files, test_targets = load_dataset('humanImages/test', subjects)

# load list of human names
human_names = [item[subjects:-1] for item in sorted(glob("humanImages/train/*/"))]

# print statistics about the dataset
print('There are %d individuals.' % len(human_names))
print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

In [ ]:
# extract pre-trained Haar Cascade face detector from cv2 library
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

In [ ]:
# function to detect the faces in an image using Haar Cascade technique
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

In [ ]:
# function to extract the face from the image supplied
def facecut(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = [[0,0,0,0]]
    faces = face_cascade.detectMultiScale(gray)
    
#returns the cordinates of the detected face or else returns [0,0,0,0]
    if type(faces) is tuple:
        return [0,0,0,0]
    else:
        return faces[0]

In [ ]:
# function to convert each image in the given path to a 4D tensor(datastructure)
def path_to_tensor(img_path):

    raw_img = cv2.imread(img_path)

#cx - is the starting x cordinate of the box around the detected face
#cy - is the starting y cordinate of the box around the detected face
#ch - height of the box around the detected face
#cx - width of the box around the detected face
    cx,cy,ch,cw = facecut(img_path)
    
    if ch != 0:
        img = cv2.resize(raw_img[cy:cy+ch,cx:cx+cw],(int(224), int(224)), interpolation = cv2.INTER_CUBIC)

#convert 2D image to 3D tensor with shape (224, 224, 3) 
        x = image.img_to_array(img)
    else:
        x = np.zeros(shape=(224, 224, 3))
        
# convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
   

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras and normalizing the pixel values by 256
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(Dropout(0.4))


model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(112,112,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(Dropout(0.4))


model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(56,56,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(Dropout(0.4))


model.add(Dense(subjects, activation='softmax'))

model.add(GlobalAveragePooling2D(data_format=None))


model.summary()

In [ ]:
# compiles the above CNN architecture with the specified learning rate(optimizer), loss function and metric a measurement
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

# number of iterations
epochs = 500

# saves the model in the folder saved_models/ with the name 'weights.best.7_12_v1'(any name) in hdf5 format
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.8_12_v0.hdf5', 
                               verbose=1, save_best_only=True)

# starts training the model with the train tensors
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

In [ ]:
# loads the saved model with the weights, this step saves us from training the model everytime
model.load_weights('saved_models/weights.best.7_12_v0_version1.hdf5')

In [38]:
# get index of predicted person for each image in test set
friend_prediction = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(friend_prediction)==np.argmax(test_targets, axis=1))/len(friend_prediction)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.8182%


In [39]:
# prints the probabilty matrix of the output layer when each of the input is predicted
for tensor in test_tensors:
    print((model.predict(np.expand_dims(tensor, axis=0))))

[[ 0.12438317  0.09723664  0.27288526  0.03357404  0.47192094]]
[[ 0.13904156  0.2179421   0.39775792  0.08742823  0.15783022]]
[[ 0.04653785  0.22481127  0.00097527  0.53150946  0.19616586]]
[[ 0.15052773  0.03183563  0.24317318  0.50212592  0.07233757]]
[[ 0.1332821   0.27653021  0.06495868  0.512402    0.01282704]]
[[ 0.10780086  0.06337125  0.25456157  0.0301108   0.54415536]]
[[ 0.09277145  0.0496951   0.53549153  0.02662375  0.29541788]]
[[ 0.13113521  0.13429679  0.30790329  0.03279104  0.39387375]]
[[ 0.11339898  0.05926939  0.52455169  0.02592628  0.27685368]]
[[ 0.17376707  0.32598165  0.30191222  0.10231743  0.09602174]]
[[ 0.07031171  0.05325769  0.00123497  0.61622888  0.25896657]]
[[ 0.29659939  0.42703155  0.01720468  0.08438451  0.17478003]]
[[ 0.13299164  0.05381297  0.49749455  0.19046481  0.12523629]]
[[ 0.20741831  0.58826035  0.01061841  0.11830077  0.07540183]]
[[ 0.11064693  0.08859558  0.31052989  0.01378215  0.47644541]]
[[ 0.14086719  0.06942822  0.40768921  0

In [ ]:
# this function converts the image to 4d tensor and forwards it to the CNN to make a prediction
def faceCNN(img_path):
    who_tensor = path_to_tensor(img_path).astype('float32')/255
    result = model.predict(who_tensor)
    print("Similarity is {}".format(result.max()))
    return human_names[np.argmax(result)]

In [ ]:
# function predicts the most similar face seen in the neural network
def face_predictor(img_path):
    temp = faceCNN(img_path)
    print("The human is similar to {}".format(temp))

In [ ]:
# 'whoisthis' folder contains all the images that are to be predicted
final_test = np.array(glob("whoisthis/*"))
for i in final_test:
    face_predictor(i)